# Taller 6 REVISION DE SIMILITUD DE DOCUMENTOS 
## RADICACION QUEJAS EN EMPRESAS DE SERVICIOS PUBLICOS


### Descripción de la data a utilizar

Los datos corresponden a quejas reales radicadas por procesos de facturación en Empresas de Servicios Públicos durante los primeros 20 días de Septiembre 2020. Se analizará la similitud de las quejas radicadas para las Empresas de Servicios Públicos: 

- VANTI: Gasera presente en Bogota, Cundinamarca, Boyacá y Santander
- CHEC: Central Hidroelectrica de Caldad 
- EMCALI: Empresas Municipales de Cali
- CELSIA: Empresa de Energia de Valle del Cauca



In [7]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Inicialmente se carga la base de datos que cuenta con las siguientes columnas:

- Nímero del  Caso: radicado que se bridó a la queja
- Tipo: Corresponde al tipo de PQR en este caso solo se seleccionaron QUEJAS
- Proceso: Corresponde a una descripción general del proceso sobre el cual se radica la queja
- Causa: Tipificación relacionada al proceso
- Filtro 2: Nombre de la empresa objeto de revisión
- Descripción: Información que radica el cliente desde su queja
    

In [108]:
data = pd.read_excel('C:/Python/NLP/nlp_projects/Clase_seis/Reporte_excel_quejas.xlsx', encoding = 'UTF-8')
data 

,Número del caso,Tipo,Proceso,Causa,Filtro 2,Descripción
0,20092220249547,QUEJA,ASESORIA Y VENTA,INADECUADA ATENCION DEL ASESOR,VANTI-GAS NATURAL,SE COMUNICA EL SEÑOR JORGE INDICANDO QUE EL JA...
1,20091720203098,QUEJA,ASESORIA Y VENTA,INADECUADA ATENCION DEL ASESOR,EMCALI,QUIEN MANIFIESTA INCONFORMIDAD POR QUE EN LA F...
2,20092320266637,QUEJA,RECLAMACIONES VIDA,DEMORA ATENCION RECLAMACION,FALABELLA,SE COMUNICA USUARIO INFORMANDO QUE HACE MAS DE...
3,20091620193646,QUEJA,EXPEDICION,COBRO A CLIENTE QUE CANCELO EL PRODUCTO,CHEC NUEVOS,SE COMUNICA INDICANDO QUE SE LE SIGUE GENERAND...
4,20091520183982,QUEJA,EXPEDICION,DEMORA EN LA CANCELACION,CHEC NUEVOS,SE COMUNICA LA SEÑORA PARA RADICAR UNA QUEJA P...
...,...,...,...,...,...,...
139,20092220246740,QUEJA,RECAUDO- CARTERA Y DESEMBOLSO,DOBLE COBRO,EPN,CLIENTE INFORMA QUE LE LLEGO DOBLE COBRO EN SU...
140,20092220246013,QUEJA,RECAUDO- CARTERA Y DESEMBOLSO,DOBLE COBRO,EPN,SOLICITA INFORMACION DE LA POLIZA POR QUE LE E...
141,20091820218489,QUEJA,EXPEDICION,COBRO A CLIENTE QUE NO TIENE SEGURO,CHEC RUN OFF,SE COMUNICA USUARIO SOLICITANDO INFORMACIÓN DE...
142,20091020135005,QUEJA,EXPEDICION,COBRO A CLIENTE QUE CANCELO EL PRODUCTO,CHEC NUEVOS,SE COMUNICA USUARIA INFORMANDO QUE HACE MAS DE...


Se seleccionan unicamente las variables necesarias

In [276]:
filtro = data.iloc[:, 4:6]
filtro.rename(columns ={'Filtro 2': 'Canal', 'Descripción': 'Queja'}, inplace = True)
filtro

,Canal,Queja
0,VANTI-GAS NATURAL,SE COMUNICA EL SEÑOR JORGE INDICANDO QUE EL JA...
1,EMCALI,QUIEN MANIFIESTA INCONFORMIDAD POR QUE EN LA F...
2,FALABELLA,SE COMUNICA USUARIO INFORMANDO QUE HACE MAS DE...
3,CHEC NUEVOS,SE COMUNICA INDICANDO QUE SE LE SIGUE GENERAND...
4,CHEC NUEVOS,SE COMUNICA LA SEÑORA PARA RADICAR UNA QUEJA P...
...,...,...
139,EPN,CLIENTE INFORMA QUE LE LLEGO DOBLE COBRO EN SU...
140,EPN,SOLICITA INFORMACION DE LA POLIZA POR QUE LE E...
141,CHEC RUN OFF,SE COMUNICA USUARIO SOLICITANDO INFORMACIÓN DE...
142,CHEC NUEVOS,SE COMUNICA USUARIA INFORMANDO QUE HACE MAS DE...


Revisamos los nombres de cada una de las empresas de servicios públicos. Algunas tienen radicaciones en diferentes "Filtro_2" por lo cual se va a realizar una agrupación, para evaluar los textos de sobre una misma empresa

In [277]:
nombres = filtro['Canal'].unique()
nombres

array(['VANTI-GAS NATURAL', 'EMCALI', 'FALABELLA', 'CHEC NUEVOS',
       'EPSA NUEVOS', 'BRILLA - GASES DE LA GUAJIRA', 'ELECTROCAQUETA',
       'AGUAS DE PEREIRA', 'BIRLLA ? GASES DEL CARIBE', 'ENERTOLIMA',
       'CHEC RUN OFF', 'COLSUBSIDIO', 'EXPERIAN AFFINNITY',
       'BRILLA - SURTIGAS', 'COMFAMILIAR ATLANTICO', 'EPN',
       'EPSA RUN OFF', 'SCOTIABANK', 'PUNTO DE VISTA',
       'CREDIVALORES LIBRANZA'], dtype=object)

Agrupamos:

In [278]:
CHEC = filtro[filtro.Canal.isin(['CHEC NUEVOS', 'CHEC RUN OFF'])]
EMCALI = filtro[filtro.Canal.isin(['EMCALI'])]
VANTI = filtro[filtro.Canal.isin(['VANTI-GAS NATURAL'])]
CELSIA = filtro[filtro.Canal.isin(['EPSA NUEVOS','EPSA RUN OFF'])]

MIRAMOS COMO SE VE UNO DE NUESTROS DATA FRAMES

In [279]:
CELSIA[0:5] # Revisamos los primeros 5 registros

,Canal,Queja
8,EPSA NUEVOS,COBRO DE UN SEGURO QUE NO HE SOLICITADO
9,EPSA NUEVOS,REFERENTE AL PAGO DE INDENIZACION DEL CASOBAN0...
10,EPSA NUEVOS,SE COMUNICA DUEÑA DE LA CASA SOLICITA CANCELAC...
11,EPSA NUEVOS,SE COMUNICA MUY MOLESTA INDICANDO QUE A LA PRO...
12,EPSA NUEVOS,SE RECIBE CASO DE MERCURIO 202000023799 TRAVÉS...


## Comenzamos el pre - procesamiento de texto

Inicialmente dejamos todos los textos de la columna queja en una sola cadena.
Hacemos este proceso para todas las empresas de servicios públicos

In [280]:
CELSIA['Queja'].values # Lista con cada una de las respuestas
CELSIA_TEXTO = " ".join(CELSIA['Queja'].values)
CELSIA_TEXTO

'COBRO DE UN SEGURO QUE NO HE SOLICITADO REFERENTE AL PAGO DE INDENIZACION DEL CASOBAN0089014752 POR ORDEN 6408998 ME PERMITO DECIRLES QUE PASARON LOS 5 DIAS Y NO SE ME HA HECHO EL PAGO PUES EN EL BANCO POPULAR NO APARECE NINGUNA CONSIGNACION POR FAVOR ME AYUDAN CORREO ENVIADO EL 10 DE SEPTIEMBRE 2.020 POR PATTE DE USTEDES SE COMUNICA DUEÑA DE LA CASA SOLICITA CANCELACIÓN Y DEVOLUCIÓN DE DINERO YA QUE NO CONOCE AL TITULAR Y LE A TOCADO PAGAR DICE QUE YA HABÍA CANCELADO OTRA PÓLIZA A NOMBRE DEL MISMO SR EN EL MES DE FEBRERO DICE QUE LE PARECE UNA FALTA DE RESPETO PÓLIZA BAN101642938 ACCIDENTES PERSONALES DE EPSA CONTRATO 3039127 SE COMUNICA MUY MOLESTA INDICANDO QUE A LA PROPIETARIA DE EL PREDIO LE SIGUE LLEGANDO EL COBRO DE LA PÓLIZA, YA HABÍA SOLICITADO LA CANCELACIÓN DESDE EL (1/07/2020 02:43 PM) Y AUN EL ESTADO DE LA PÓLIZA REFLEJA COMO ACTIVA, SOLICITA QUE SE CANCELE LA PÓLIZA YA QUE HA TENIDO PROBLEMAS CON LA DUEÑA DE LA CASA BAN101669130 ACCIDENTES PERSONALES EAP SE RECIBE CASO D

In [281]:
CHEC['Queja'].values # Lista con cada una de las respuestas
CHEC_TEXTO = " ".join(CHEC['Queja'].values)
CHEC_TEXTO

'SE COMUNICA INDICANDO QUE SE LE SIGUE GENERANDO COBRO DE LA PÓLIZA QUE CANCELO EL DÍA 27/06/2020 11:41 AM,LA POLIZA YA REGISTRA CANCELADA BAN103510971 SE COMUNICA LA SEÑORA PARA RADICAR UNA QUEJA POR DEMORA EN LA CANCELACIÓN YA QUE SE LE SIGUE GENERANDO COBRO BAN 101017029 ACCIDENTES PERSONALES Y SE EVIDENCIA QUE LA PÓLIZA ESTA ACTIVA SE COMUNICA USUARIO INFORMANDO QUE NO LE HAN REALIZADO LA DEVOLUCIÓN QUE LE HABÍAN INDICADO QUE LE IBAN A REALIZAR EN LA FACTURA DE LA CHEC, POR LO QUE REQUIERE INFORMACIÓN DEL POR QUE DE LA DEMORA Y DESEA UNA RESPUESTA LO MAS PRONTO POSIBLE. USUARIO SOLICITA LA DEVOLUCIÓN DE LOS COBROS GENERADOS POR LA PÓLIZA BAN101029255 COBRADA A TRAVÉS DE CHEC, DADO QUE LA MAMA FALLECIÓ EN EL 2019 Y A LA FECHA LE SIGUEN LLEGANDO LOS COBROS, ES DE ACLARAR QUE EL NUNCA HIZO RECLAMACIÓN DADO QUE LA PÓLIZA SOLO CUBRÍA POR ACCIDENTE. SE COMUNICA QUIEN PAGA LOS SERVICIOS INFORMA QUE LE ESTÁN COBRANDO UNA PÓLIZA LA CUAL ELLA NO AUTORIZO TAMPOCO CONOCE LA TITULAR DESEA QUE L

In [282]:
VANTI['Queja'].values # Lista con cada una de las respuestas
VANTI_TEXTO = " ".join(VANTI['Queja'].values)
VANTI_TEXTO

'SE COMUNICA EL SEÑOR JORGE INDICANDO QUE EL JAMAS TOMO ESTE SEGURO BAN 100922383 VIDA QUE SI EL HUBIERA DADO SU AUTORIZACIÓN TODO HUBIERA SIDO DIFERENTE EL DESEA QUE LE EXPLIQUEN PORQUE LE APARECIÓ ESTE SEGURO EN LA FACTURA DE VANTI SUSCRIPTOR 26986854 Y DESEA LA DEVOLUCIÓN DEL DINERO SE COMUNICA USUARIO INFORMANDO QUE HACE MAS DE DOS MESES QUE SOLICITO LA CANCELACIÓN DE LA PÓLIZA QUE TOMO POR MEDIO DE LAS FACTURAS DEL GAS NATURAL, POR LO QUE NO ENTIENDE EL POR QUE EL SEGURO SIGUE GENERANDO COBRO SIENDO QUE YA SE CANCELO, POR LO QUE DESEA QUE SE LE GENERE LA CANCELACIÓN DE LA PÓLIZA Y LA ANULACIÓN DEL COBRO. SE COMUNICA LA USUARIA ANA REYES INFORMANDO QUE EN LA FACTURA LE ESTA LLEGANDO UN COBRO DE UN SEGURO EL CUAL ELLA EN NINGÚN MOMENTO TOMO Y TAMPOCO DIO AUTORIZACION PARA QUE SE LE ESTUVIERA GENERANDO EL COBRO EN LA FACTURA DEL GAS NATURAL, POR LO QUE REQUIERE QUE SE LE CANCELE EL PRODUCTO Y SE LE ANULEN LOS COBROS EN LA FACTURA YA QUE ELLA NO ESTA DISPUESTA A PAGAR SE COMUNICA EL S

In [283]:
EMCALI['Queja'].values # Lista con cada una de las respuestas
EMCALI_TEXTO = " ".join(EMCALI['Queja'].values)
EMCALI_TEXTO

'QUIEN MANIFIESTA INCONFORMIDAD POR QUE EN LA FACTURA DEL MES DE JULIO(PARA PAGAR EN EL MES DE AGOSTO) LLEGO UN COBRO DE ALIANZA EMCALI, YO NO HE AUTORIZADO ESE COBRO. SE COMUNICA YA QUE AUN LE SIGUEN COBRANDO LAS PÓLIZAS BAN 100371753 ACCIDENTES PERSONALES Y BAN 100413078 VIDA YA QUE EL LLAMO A CANCELARLAS SEGÚN EL CASO 20071419547478 INDICA QUE JAMAS HA QUEDADO EN MORA CON EL PAGO DE LAS PÓLIZAS Y DESEA QUE LE RESPONDAN POR ESTO CLIENTE INFORMA QUE NUNCA LE AVISARON QUE LA PÓLIZA ESTABA CANCELADA Y ADICIONAL MENTE SE LA SIGUEN COBRANDO CLIENTE INFORMA QUE QUIERE SEGUIR CON ELLA Y TAMBIÉN POR QUE FUE CANCELADA ENVIA FACTURAS DONDE SE EVIDENCIA EL COBRO SE ADJUNTAN AL CASO PÓLIZA BAN100371795 ACCIDENTES PERSONALES CONTRATO 571586 DE EMCALI LA SEÑORA NURY INDICA QUE ELLA JAMAS TOMO LAS PÓLIZAS BAN 100421266 ACCIDENTES PERSONALES Y BAN 100413455 VIDA Y MENOS CON SURAMERICANA QUE ELLA NO LO TOMO Y DESEA SABER PORQUE HAY DOS SEGUROS A NOMBRE DE ELLA SE COMUNICA PARA VERIFICAR PORQUE LE EST

## Continuamos con el pre-procesamiento hasta llegar al steeming de cada documento (string)

### PREPROCESAMIENTO CELSIA

In [284]:
CELSIA_TEXTO = CELSIA_TEXTO.lower()
CELSIA_TEXTO = re.sub(r"[\W\d_]+"," ",CELSIA_TEXTO)
CELSIA_TEXTO = CELSIA_TEXTO.strip()
CELSIA_TEXTO = CELSIA_TEXTO.split()

In [285]:
for line in CELSIA_TEXTO:
    CELSIA_TEXTO = [palabra for palabra in CELSIA_TEXTO if palabra not in stopwords_sp]
    break

In [286]:
CELSIA_FINAL = []

for palabra in CELSIA_TEXTO:
    CELSIA_STEM = spanishStemmer.stem(palabra)
    CELSIA_FINAL.append(CELSIA_STEM)

CELSIA_FINAL = " ".join(CELSIA_FINAL)
CELSIA_FINAL

'cobr segur solicit referent pag indenizacion casob orden permit dec pas dias hech pag pues banc popul aparec ningun consignacion favor ayud corre envi septiembr patt usted comun dueñ cas solicit cancel devolu diner conoc titul toc pag dic cancel poliz nombr mism sr mes febrer dic parec falt respet poliz ban accident personal epsa contrat comun molest indic propietari predi sig lleg cobr poliz solicit cancel pm aun poliz reflej activ solicit cancel poliz problem dueñ cas ban accident personal eap recib cas mercuri traves epsa baj nic permit envi document requier retir cobr empres sur cancel cost sab adquir autoriz igual devuelv diner ven cobr recib servici energ comprob pattycvac gmail com comun usuari inform gener cobr segur ningun moment tom tampoc dio autorizacion vent product requier gener cancel poliz devolu diner cobr part poliz suscriptor poliz ban vid anticip enfermedad grav comun señor indic hac dos años pag poliz epsa baj contrat valor evidenci señor ningun product compañ ind

### PREPROCESAMIENTO CHEC

In [287]:
CHEC_TEXTO = CHEC_TEXTO.lower()
CHEC_TEXTO = re.sub(r"[\W\d_]+"," ",CHEC_TEXTO)
CHEC_TEXTO = CHEC_TEXTO.strip()
CHEC_TEXTO = CHEC_TEXTO.split()

In [288]:
for line in CHEC_TEXTO:
    CHEC_TEXTO = [palabra for palabra in CHEC_TEXTO if palabra not in stopwords_sp]
    break

In [289]:
CHEC_FINAL = []

for palabra in CHEC_TEXTO:
    CHEC_STEM = spanishStemmer.stem(palabra)
    CHEC_FINAL.append(CHEC_STEM)

CHEC_FINAL = " ".join(CHEC_FINAL)
CHEC_FINAL

'comun indic sig gener cobr poliz cancel dia am poliz registr cancel ban comun señor radic quej demor cancel sig gener cobr ban accident personal evident poliz activ comun usuari inform realiz devolu indic iban realiz factur chec requier inform demor dese respuest mas pront posibl usuari solicit devolu cobr gener poliz ban cobr traves chec dad mam fallec fech sig lleg cobr aclar nunc hiz reclam dad poliz sol cubr accident comun pag servici inform cobr poliz autoriz tampoc conoc titul dese retir cobr acumul siempr pag consum dispuest pag poliz encuentr cancel fech juli cobr mor poliz ban accident personal contrat chec comun usuari inform mes febrer solicit cancel poliz registr factur chec indic poliz encontr suspend realiz anul cobr despues notif sigu lleg cobr factur encuentr conform des respuest mas pront posibl suscriptor comun sab hech pag dich iban hac pag mas tard dias habil banc ayer aparec todav pag exp ban client comun sab cancel poliz ban vid contrat chec inform ped cancel nin

### PREPROCESAMIENTO VANTI

In [290]:
VANTI_TEXTO = VANTI_TEXTO.lower()
VANTI_TEXTO = re.sub(r"[\W\d_]+"," ",VANTI_TEXTO)
VANTI_TEXTO = VANTI_TEXTO.strip()
VANTI_TEXTO = VANTI_TEXTO.split()

In [291]:
for line in VANTI_TEXTO:
    VANTI_TEXTO = [palabra for palabra in VANTI_TEXTO if palabra not in stopwords_sp]
    break

In [292]:
VANTI_FINAL = []

for palabra in VANTI_TEXTO:
    VANTI_STEM = spanishStemmer.stem(palabra)
    VANTI_FINAL.append(VANTI_STEM)

VANTI_FINAL = " ".join(VANTI_FINAL)
VANTI_FINAL

'comun señor jorg indic jam tom segur ban vid si dad autoriz sid diferent dese expliqu aparec segur factur vanti suscriptor dese devolu diner comun usuari inform hac mas dos mes solicit cancel poliz tom medi factur gas natural entiend segur sig gener cobr siend cancel dese gener cancel poliz anul cobr comun usuari ana rey inform factur lleg cobr segur ningun moment tom tampoc dio autorizacion gener cobr factur gas natural requier cancel product anul cobr factur dispuest pag comun señor indic aun gener cobr poliz cancel val sistem poliz ban vid aun encuentr activ señor exig quit valor factur dig gas orient ningun vincul señor quier favor colabor recib quej asegur indic hab autoriz poliz dese cancel suscriptor poliz ban vid recib solicitud radic gaser asegur indic inform segur gratuit inconform dese cancel suscriptor poliz ban vid comun usuari inform factur gas natural lleg cobr segur ningun moment tom tampoc dio autorizacion gener cobr factur requier gener cancel poliz anul cobr gener f

### PREPROCESAMIENTO EMCALI

In [293]:
EMCALI_TEXTO = EMCALI_TEXTO.lower()
EMCALI_TEXTO = re.sub(r"[\W\d_]+"," ",EMCALI_TEXTO)
EMCALI_TEXTO = EMCALI_TEXTO.strip()
EMCALI_TEXTO = EMCALI_TEXTO.split()

In [294]:
for line in EMCALI_TEXTO:
    EMCALI_TEXTO = [palabra for palabra in EMCALI_TEXTO if palabra not in stopwords_sp]
    break

In [295]:
EMCALI_FINAL = []

for palabra in EMCALI_TEXTO:
    EMCALI_STEM = spanishStemmer.stem(palabra)
    EMCALI_FINAL.append(EMCALI_STEM)

EMCALI_FINAL = " ".join(EMCALI_FINAL)
EMCALI_FINAL

'manifiest inconform factur mes juli pag mes agost lleg cobr alianz emcali autoriz cobr comun aun sig cobr poliz ban accident personal ban vid llam cancel segun cas indic jam qued mor pag poliz dese respond client inform nunc avis poliz cancel adicional ment sig cobr client inform quier segu cancel envi factur evident cobr adjunt cas poliz ban accident personal contrat emcali señor nury indic jam tom poliz ban accident personal ban vid men suramerican tom dese sab dos segur nombr comun verific lleg aun cobr poliz ban accident personal si cancel segun cas verif poliz cancel comun indic lleg cobr aun poliz ban vid suscriptor client dese devolu diner retir cobr poliz ban vid factur numer emcali encuentr cancel sig lleg cobr llam pag servici envi factur corre indic nunc autoriz cobr solicit quej error expedicion cobr poliz papà falleci agost hac años muert natural ban plan accident personal ban plan vid suscriptor poliz emcali hij exigu devolucion diner pag cancelacion poliz inmediat aun e

**Unimos los textos preporcesados en un Pandas DataFrame**

In [297]:
empresa = {'Empresa': ['EMCALI', 'VANTI', 'CHEC', 'CELSIA'],
          'Queja_procesada': [EMCALI_FINAL, VANTI_FINAL, CHEC_FINAL, CELSIA_FINAL]}

empresa = pd.DataFrame(empresa, columns = ['Empresa', 'Queja_procesada'])
empresa

,Empresa,Queja_procesada
0,EMCALI,manifiest inconform factur mes juli pag mes ag...
1,VANTI,comun señor jorg indic jam tom segur ban vid s...
2,CHEC,comun indic sig gener cobr poliz cancel dia am...
3,CELSIA,cobr segur solicit referent pag indenizacion c...


# SE GENERA LA MATRIZ TF-IDF



In [302]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(empresa.Queja_procesada.values)

tfidf_matrix = pd.DataFrame(data=tfidf.toarray(), columns=tfidf_vect.get_feature_names())

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.columns = empresa.Empresa.values

tfidf_matrix

,EMCALI,VANTI,CHEC,CELSIA
abril,0.000,0.000,0.026,0.000
abus,0.000,0.016,0.000,0.000
accident,0.089,0.000,0.168,0.163
acept,0.000,0.021,0.017,0.027
acerc,0.016,0.000,0.000,0.067
...,...,...,...,...
vincul,0.000,0.016,0.000,0.000
viv,0.000,0.013,0.021,0.000
volv,0.000,0.016,0.000,0.000
vulner,0.040,0.000,0.000,0.000


## Calculo de la Distancia del coseno


In [303]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

,EMCALI,VANTI,CHEC,CELSIA
EMCALI,0.000000,0.546466,0.539687,0.551773
VANTI,0.546466,0.000000,0.248157,0.370509
CHEC,0.539687,0.248157,0.000000,0.353936
CELSIA,0.551773,0.370509,0.353936,0.000000


### CONCLUSIONES GENERALES DE LA DISTANCIA DEL COSENO

EMCALI, VANTI Y CELSIA tienen radicaciones de quejas similares a CHEC.

LAS QUEJAS MAS SIMILARES SE PRESENTAN EN VANTI Y CHEC con lo cual se podria comezar a inferir que los problemas que se presentan en estas dos empresas de servicios públicos son similares. 
 
Contrario a CHEC y VANTI las quejas de CELSIA Y EMCALI son las que mas difieren; por lo que las radicaciones de quejas entre estas dos empresas difieren en cierta medida.